In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
tk = th.Thicket.from_caliperreader("./calipers")

(2/2) Creating Thicket: 100%|██████████| 267/267 [00:00<00:00, 291.22it/s]
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/thicket/ensemble.py:409: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace({numerical_fill_value: None}, inplace=True)


In [3]:
tk.show_metric_columns()

['Max time/rank (exc)',
 'Calls/rank (avg)',
 'Variance time/rank',
 'Min time/rank',
 'Avg time/rank (exc)',
 'Calls/rank (min)',
 'Calls/rank (total)',
 'Max time/rank',
 'Total time (exc)',
 'Avg time/rank',
 'Min time/rank (exc)',
 'Calls/rank (max)',
 'Total time']

In [4]:
metadata_df = tk.metadata.copy()

metadata_df['input_type'] = metadata_df.apply(lambda x: x['cmdline'][2], axis=1)
tk.metadata = metadata_df.copy()

In [5]:
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("input_type")

In [6]:
input_sizes = sorted(tk.metadata["input_size"].unique())
num_procs = sorted(tk.metadata["num_procs"].unique())
input_types = sorted(tk.metadata["input_type"].unique())

In [7]:
print(tk.tree(metric_column="Avg time/rank"))

# tk.dataframe

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

1.919 main
├─ 0.004 MPI_Comm_dup
├─ 0.000 MPI_Finalize
├─ 0.000 MPI_Finalized
├─ 0.000 MPI_Init
├─ 0.000 MPI_Initialized
├─ 0.051 comm
│  ├─ 0.037 MPI_Barrier
│  └─ 0.013 comm_large
│     ├─ 0.009 MPI_Recv
│     └─ 0.004 MPI_Send
├─ 0.002 comp
│  ├─ 0.001 comp_large
│  └─ 0.000 comp_small
├─ 0.000 correctness_check
│  ├─ 0.000 MPI_Recv
│  └─ 0.000 MPI_Send
└─ 0.000 data_init_local

Legend (Metric: Avg time/rank Min: 0.00 Max: 1.92 indices: {'profile': np.int64(14471934)})
█ 1.73 - 1.92
█ 1.34 - 1.73
█ 0.96 - 1.34
█ 0.58 - 0.96
█ 0.19 - 0.58
█ 0.00 - 0.19

name User code    ◀  Only in left graph    ▶  Only in right graph



# Analysis

In [8]:
import shutil
import os

shutil.rmtree('plots')
os.mkdir('plots')

## Strong Scaling

These graphs will show average time/rank for every processor size for a select number of input sizes, highlighting how the time/rank changes as the number of processors increases.

### Avg time/rank

In [9]:
interesting_input_sizes = [65536, 16777216, 67108864, 268435456]

x_ticks = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

plt.figure(figsize=(10, 6))

for input_size in interesting_input_sizes:
    x = []
    y_avg = []
    y_min = []
    y_max = []

    for num_procs in x_ticks:
        avg_times = []

        for main_node in tk.graph.traverse():
            if main_node.frame['name'] != 'main':
                continue

            data = tk.dataframe.loc[main_node]

            if not isinstance(data, pd.DataFrame):
                data = pd.DataFrame([data])

            subset = data[
                (data['num_procs'] == num_procs) &
                (data['input_size'] == input_size)
                ]

            if subset.empty:
                continue

            avg_times.extend(subset['Avg time/rank'].values)

        if not avg_times:
            continue
            
        avg_time = np.mean(avg_times)
        min_time = np.min(avg_times)
        max_time = np.max(avg_times)

        x.append(num_procs)
        y_avg.append(avg_time)
        y_min.append(min_time)
        y_max.append(max_time)

    if not x:
        continue

    x = np.array(x)
    y_avg = np.array(y_avg)
    y_min = np.array(y_min)
    y_max = np.array(y_max)

    plt.plot(x, y_avg, marker='o', label=f'Input Size={input_size}')

    plt.fill_between(x, y_min, y_max, alpha=0.2)

plt.title('Strong Scaling: Avg Time per Rank vs Number of Processes\n(Averaged over Input Types)')
plt.xlabel('Number of Processes')
plt.ylabel('Avg Time per Rank (seconds)')

plt.xscale('log', base=2)
plt.xticks(x_ticks)
plt.grid(True, which='both', linestyle='--', linewidth=0.7)

plt.legend(title='Input Size')
plt.tight_layout()

plt_filename = './plots/strong_scaling_avg_across_input_types.png'
plt.savefig(plt_filename, format='png', bbox_inches='tight')
plt.close()


### Total time

In [10]:
plt.figure(figsize=(10, 6))

for input_size in interesting_input_sizes:
    x = []
    y_avg = []
    y_min = []
    y_max = []

    for num_procs in x_ticks:
        avg_times = []

        for main_node in tk.graph.traverse():
            if main_node.frame['name'] != 'main':
                continue

            data = tk.dataframe.loc[main_node]

            if not isinstance(data, pd.DataFrame):
                data = pd.DataFrame([data])

            subset = data[
                (data['num_procs'] == num_procs) &
                (data['input_size'] == input_size)
                ]

            if subset.empty:
                continue

            avg_times.extend(subset['Total time'].values)

        if not avg_times:
            continue

        avg_time = np.mean(avg_times)
        min_time = np.min(avg_times)
        max_time = np.max(avg_times)

        x.append(num_procs)
        y_avg.append(avg_time)
        y_min.append(min_time)
        y_max.append(max_time)

    if not x:
        continue

    x = np.array(x)
    y_avg = np.array(y_avg)
    y_min = np.array(y_min)
    y_max = np.array(y_max)

    plt.plot(x, y_avg, marker='o', label=f'Input Size={input_size}')

    plt.fill_between(x, y_min, y_max, alpha=0.2)

plt.title('Strong Scaling: Total Time vs Number of Processes\n(Averaged over Input Types)')
plt.xlabel('Number of Processes')
plt.ylabel('Total time (seconds)')

plt.xscale('log', base=2)
plt.xticks(x_ticks)
plt.grid(True, which='both', linestyle='--', linewidth=0.7)

plt.legend(title='Input Size')
plt.tight_layout()

plt_filename = './plots/strong_scaling_avg_across_input_types_total_time.png'
plt.savefig(plt_filename, format='png', bbox_inches='tight')
plt.close()


# Weak Scaling Analysis

These plots will double input size as processor size doubles, showing weak scaling

### Avg time / rank

In [11]:
num_procs_list = [2**i for i in range(1, 11, 2)]
input_size_per_proc = 65536 * 2
input_sizes_list = [2**i for i in range(16, 29, 2)]

procs_sizes_pairs = list(zip(num_procs_list, input_sizes_list))

x = []
y_avg = []
y_min = []
y_max = []

for num_procs, input_size in procs_sizes_pairs:
    avg_times = []

    for main_node in tk.graph.traverse():
        if main_node.frame['name'] != 'main':
            continue

        data = tk.dataframe.loc[main_node]

        if not isinstance(data, pd.DataFrame):
            data = pd.DataFrame([data])

        subset = data[
            (data['num_procs'] == num_procs) &
            (data['input_size'] == input_size)
            ]

        if subset.empty:
            continue

        avg_times.extend(subset['Avg time/rank'].values)

    if not avg_times:
        continue

    avg_time = np.mean(avg_times)
    min_time = np.min(avg_times)
    max_time = np.max(avg_times)

    x.append(num_procs)
    y_avg.append(avg_time)
    y_min.append(min_time)
    y_max.append(max_time)

x = np.array(x)
y_avg = np.array(y_avg)
y_min = np.array(y_min)
y_max = np.array(y_max)

num_procs_exponents = np.log2(x).astype(int)
input_size_exponents = np.log2(x * input_size_per_proc).astype(int)

plt.figure(figsize=(10, 6))
plt.plot(x, y_avg, marker='o', label='Avg Time per Rank', color='blue')

plt.fill_between(x, y_min, y_max, color='blue', alpha=0.2)

plt.title('Weak Scaling: Avg Time per Rank vs Processor Count and Input Size\n(Averaged over Input Types)')
plt.xlabel('Number of Processes\n(Input Size)')
plt.ylabel('Avg Time per Rank (seconds)')

plt.xscale('log', base=2)

x_tick_labels = [
    f'$2^{{{n_exp}}}$\n($2^{{{s_exp}}}$)' for n_exp, s_exp in zip(num_procs_exponents, input_size_exponents)
]
plt.xticks(x, labels=x_tick_labels)
plt.grid(True, which='both', linestyle='--', linewidth=0.7)

plt.tight_layout()

plt_filename = './plots/weak_scaling_avg_across_input_types.png'
plt.savefig(plt_filename, format='png', bbox_inches='tight')
plt.close()


### Total Time

In [12]:
num_procs_list = [2**i for i in range(1, 11, 2)]
input_size_per_proc = 65536 * 2
input_sizes_list = [2**i for i in range(16, 29, 2)]

procs_sizes_pairs = list(zip(num_procs_list, input_sizes_list))

x = []
y_avg = []
y_min = []
y_max = []

for num_procs, input_size in procs_sizes_pairs:
    avg_times = []

    for main_node in tk.graph.traverse():
        if main_node.frame['name'] != 'main':
            continue

        data = tk.dataframe.loc[main_node]

        if not isinstance(data, pd.DataFrame):
            data = pd.DataFrame([data])

        subset = data[
            (data['num_procs'] == num_procs) &
            (data['input_size'] == input_size)
            ]

        if subset.empty:
            continue

        avg_times.extend(subset['Total time'].values)

    if not avg_times:
        continue

    avg_time = np.mean(avg_times)
    min_time = np.min(avg_times)
    max_time = np.max(avg_times)

    x.append(num_procs)
    y_avg.append(avg_time)
    y_min.append(min_time)
    y_max.append(max_time)

x = np.array(x)
y_avg = np.array(y_avg)
y_min = np.array(y_min)
y_max = np.array(y_max)

num_procs_exponents = np.log2(x).astype(int)
input_size_exponents = np.log2(x * input_size_per_proc).astype(int)

plt.figure(figsize=(10, 6))
plt.plot(x, y_avg, marker='o', label='Avg Time per Rank', color='blue')

plt.fill_between(x, y_min, y_max, color='blue', alpha=0.2)

plt.title('Weak Scaling: Total Time vs Processor Count and Input Size\n(Averaged over Input Types)')
plt.xlabel('Number of Processes\n(Input Size)')
plt.ylabel('Total Time (seconds)')

plt.xscale('log', base=2)

x_tick_labels = [
    f'$2^{{{n_exp}}}$\n($2^{{{s_exp}}}$)' for n_exp, s_exp in zip(num_procs_exponents, input_size_exponents)
]
plt.xticks(x, labels=x_tick_labels)
plt.grid(True, which='both', linestyle='--', linewidth=0.7)

plt.tight_layout()

plt_filename = './plots/weak_scaling_avg_across_input_types_total_time.png'
plt.savefig(plt_filename, format='png', bbox_inches='tight')
plt.close()


## Input Type Impact on Performance

In [13]:
import seaborn as sns

os.makedirs('plots', exist_ok=True)

input_types = ['Sorted', 'Random', 'ReverseSorted', '1_perc_perturbed']

data_list = []

for node in tk.graph.traverse():
    data = tk.dataframe.loc[node]
    if node.frame['name'] != 'main':
        continue

    for input_type in input_types:
        data2 = data[data['input_type'] == input_type]

        for value in data2['Avg time/rank']:
            data_list.append({
                'input_type': input_type,
                'avg_time_per_rank': value
            })

df = pd.DataFrame(data_list)

def remove_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    return data[(data >= Q1 - 1.5 * IQR) & (data <= Q3 + 1.5 * IQR)]

df_clean = df.groupby('input_type')['avg_time_per_rank'].apply(remove_outliers).reset_index(level=0)

plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df_clean,
    x='input_type',
    y='avg_time_per_rank',
    palette=['blue', 'green', 'red', 'purple'],
    showfliers=False,
    boxprops=dict(edgecolor='black')
)

plt.title('Impact of Input Type on Performance (Outliers Removed)', fontsize=16)
plt.xlabel('Input Type', fontsize=12)
plt.ylabel('Average Time per Rank (seconds)', fontsize=12)

plt.grid(axis='y', linestyle='--', linewidth=0.7)

plt.tight_layout()
plt_filename = './plots/input_type_impact_on_performance.png'
plt.savefig(plt_filename, format='png', bbox_inches='tight')
plt.close()


/var/folders/cv/bhp4208d2zn_l86xn9h3hhn40000gn/T/ipykernel_37190/1652339275.py:34: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(


## Communication Overhead

In [14]:
comm_data = []

num_procs_list = sorted(tk.metadata["num_procs"].unique())
input_sizes = tk.metadata["input_size"].unique()
input_types = tk.metadata["input_type"].unique()

communication_nodes = [
    'MPI_Barrier',
    'MPI_Recv',
    'MPI_Send',
    'comm',
    'comm_large',
]

for main_node in tk.graph.traverse():
    if main_node.frame['name'] != 'main':
        continue

    data = tk.dataframe.loc[main_node]

    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame([data])

    for input_size in input_sizes:
        for input_type in input_types:
            for num_procs in num_procs_list:
                data_filtered = data[
                    (data['input_size'] == input_size) &
                    (data['input_type'] == input_type) &
                    (data['num_procs'] == num_procs)
                    ]

                if data_filtered.empty:
                    continue

                data_row = data_filtered.iloc[0]

                total_time = data_row['Total time']

                comm_time = 0.0

                for node in main_node.traverse():
                    node_name = node.frame['name']
                    if node_name in communication_nodes:
                        node_data = tk.dataframe.loc[node]

                        if not isinstance(node_data, pd.DataFrame):
                            node_data = pd.DataFrame([node_data])

                        node_data_filtered = node_data[
                            (node_data['input_size'] == input_size) &
                            (node_data['input_type'] == input_type) &
                            (node_data['num_procs'] == num_procs)
                            ]

                        if node_data_filtered.empty:
                            continue

                        node_data_row = node_data_filtered.iloc[0]

                        comm_time += node_data_row['Total time']

                comm_overhead = (comm_time / total_time) * 100

                comm_data.append({
                    'num_procs': num_procs,
                    'input_size': input_size,
                    'input_type': input_type,
                    'comm_overhead': comm_overhead,
                })

comm_df = pd.DataFrame(comm_data)

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt

comm_df['num_procs'] = comm_df['num_procs'].astype(int)
comm_df['input_size'] = comm_df['input_size'].astype(int)
comm_df['input_type'] = comm_df['input_type'].astype(str)

g = sns.FacetGrid(
    comm_df,
    col='input_type',
    row='input_size',
    margin_titles=True,
    height=3,
    aspect=1.5
)

g.map_dataframe(
    sns.lineplot,
    x='num_procs',
    y='comm_overhead',
    estimator='mean',
    ci='sd',
    marker='o',
    err_style='band'
)

for ax in g.axes.flat:
    ax.set_xscale('log', base=2)
    ax.set_xticks(num_procs_list)
    ax.grid(True, which='both', linestyle='--', linewidth=0.7)
    ax.set_xlabel('Number of Processes')
    ax.set_ylabel('Communication Overhead (%)')

g.set_titles(col_template="{col_name}", row_template="Input Size: {row_name}")

plt.tight_layout()

g.fig.subplots_adjust(top=0.92)

g.fig.suptitle('Communication Overhead vs Number of Processes', fontsize=16, y=0.95)

plt.savefig('./plots/communication_overhead_facetgrid.png', bbox_inches='tight')
plt.close()


/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/L

## MPI_Barrier Overhead

In [16]:
barrier_data = []

num_procs_list = sorted(tk.metadata["num_procs"].unique())
input_sizes = tk.metadata["input_size"].unique()
input_types = tk.metadata["input_type"].unique()

communication_nodes = [
    'MPI_Barrier',
    'MPI_Recv',
    'MPI_Send',
    'comm',
    'comm_large',
]

barrier_node = 'MPI_Barrier'

for main_node in tk.graph.traverse():
    if main_node.frame['name'] != 'main':
        continue

    data = tk.dataframe.loc[main_node]

    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame([data])

    for input_size in input_sizes:
        for input_type in input_types:
            for num_procs in num_procs_list:
                data_filtered = data[
                    (data['input_size'] == input_size) &
                    (data['input_type'] == input_type) &
                    (data['num_procs'] == num_procs)
                    ]

                if data_filtered.empty:
                    continue

                total_comm_time = 0.0
                mpi_barrier_time = 0.0

                for node in main_node.traverse():
                    node_name = node.frame['name']
                    node_data = tk.dataframe.loc[node]

                    if not isinstance(node_data, pd.DataFrame):
                        node_data = pd.DataFrame([node_data])

                    node_data_filtered = node_data[
                        (node_data['input_size'] == input_size) &
                        (node_data['input_type'] == input_type) &
                        (node_data['num_procs'] == num_procs)
                        ]

                    if node_data_filtered.empty:
                        continue

                    node_data_row = node_data_filtered.iloc[0]
                    node_time = node_data_row['Total time']

                    if node_name in communication_nodes:
                        total_comm_time += node_time
                        if node_name == barrier_node:
                            mpi_barrier_time += node_time

                if total_comm_time == 0:
                    barrier_percentage = 0.0
                else:
                    barrier_percentage = (mpi_barrier_time / total_comm_time) * 100

                barrier_data.append({
                    'num_procs': num_procs,
                    'input_size': input_size,
                    'input_type': input_type,
                    'barrier_percentage': barrier_percentage,
                })

barrier_df = pd.DataFrame(barrier_data)

In [17]:
barrier_df['num_procs'] = barrier_df['num_procs'].astype(int)
barrier_df['input_size'] = barrier_df['input_size'].astype(int)
barrier_df['input_type'] = barrier_df['input_type'].astype(str)

g = sns.FacetGrid(
    barrier_df,
    col='input_type',
    row='input_size',
    margin_titles=True,
    height=3.5,
    aspect=1.5
)

g.map_dataframe(
    sns.lineplot,
    x='num_procs',
    y='barrier_percentage',
    estimator='mean',
    ci='sd',
    marker='o',
    err_style='band'
)

for ax in g.axes.flat:
    ax.set_xscale('log', base=2)
    ax.set_xticks(num_procs_list)
    ax.grid(True, which='both', linestyle='--', linewidth=0.7)
    ax.set_xlabel('Number of Processes')
    ax.set_ylabel('Barrier Percentage of Communication Time (%)')

g.set_titles(col_template="{col_name}", row_template="Input Size: {row_name}")

plt.tight_layout()

g.fig.subplots_adjust(top=0.92)

g.fig.suptitle('Percentage of Communication Time Spent in MPI_Barrier', fontsize=16, y=0.95)

plt.savefig('./plots/mpi_barrier_percentage_facetgrid.png', bbox_inches='tight')
plt.close()


/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/Lab2/analysis/.venv/lib/python3.12/site-packages/seaborn/axisgrid.py:854: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  func(*plot_args, **plot_kwargs)
/Users/jacksonstone/Documents/CSCE 435/L

In [55]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

algorithm = "Bitonic Sort"

os.makedirs('plots_report', exist_ok=True)

input_sizes = [2**16, 2**18, 2**20, 2**22, 2**24, 2**26, 2**28]
input_types = ['Sorted', 'Random', 'ReverseSorted', '1_perc_perturbed']
num_procs_list = sorted(tk.metadata["num_procs"].unique())

all_data = []

for node in tk.graph.traverse():
    data = tk.dataframe.loc[node]
    if node.frame['name'] not in ['comp_large', 'comm', 'main']:
        continue

    for input_size in input_sizes:
        for input_type in input_types:
            for num_procs in num_procs_list:
                subset = data[
                    (data['input_size'] == input_size) &
                    (data['input_type'] == input_type) &
                    (data['num_procs'] == num_procs)
                    ]
                if not subset.empty:
                    row = subset.iloc[0]
                    all_data.append({
                        'node': node.frame['name'],
                        'input_size': input_size,
                        'input_type': input_type,
                        'num_procs': num_procs,
                        'total_time': row['Total time'],
                        'avg_time_per_rank': row['Avg time/rank']
                    })

df = pd.DataFrame(all_data)

for node_name in ['comp_large', 'comm', 'main']:
    for input_size in input_sizes:
        plt.figure(figsize=(10, 6))
        subset = df[(df['node'] == node_name) & (df['input_size'] == input_size)]
        
        for input_type, group in subset.groupby('input_type'):
            sns.lineplot(x='num_procs', y='avg_time_per_rank', data=group, marker='o', label=f'Input Type: {input_type}')
            
        plt.xscale('log', base=2)
        
        plt.xticks(num_procs_list)
        plt.grid(True, which='both', linestyle='--', linewidth=0.7)
        plt.title(f'{algorithm}\nStrong Scaling: {node_name} - Input Size: {input_size}')
        plt.xlabel('Number of Processes')
        plt.ylabel('Average Time per Rank (seconds)')
        plt.legend(title='Input Type')
        plt.tight_layout()
        plt.savefig(f'./plots_report/strong_scaling_{node_name}_{input_size}.png')
        plt.close()
        
for node_name in ['comp_large', 'comm', 'main']:
    for input_type in input_types:
        plt.figure(figsize=(10, 6))
        subset = df[(df['node'] == node_name) & (df['input_type'] == input_type)]

        for input_size, group in subset.groupby('input_size'):
            group = group.copy()

            baseline = group[group['num_procs'] == 2]
            if baseline.empty:
                continue

            T1 = baseline['avg_time_per_rank'].iloc[0] * 2

            group['speedup'] = T1 / group['avg_time_per_rank']

            sns.lineplot(x='num_procs', y='speedup', data=group, marker='o', label=f'Input Size: {input_size}')

        plt.xscale('log', base=2)
        plt.yscale('log', base=2)
        plt.xticks(num_procs_list)
        plt.grid(True, which='both', linestyle='--', linewidth=0.7)
        plt.title(f'{algorithm}\nStrong Scaling Speedup: {node_name} - {input_type}')
        plt.xlabel('Number of Processes')
        plt.ylabel('Speedup')
        plt.legend(title='Input Size')
        plt.tight_layout()
        plt.savefig(f'./plots_report/speedup_{node_name}_{input_type}.png')
        plt.close()

weak_scaling_data = []

for node in tk.graph.traverse():
    data = tk.dataframe.loc[node]
    if node.frame['name'] not in ['comp_large', 'comm', 'main']:
        continue

    for num_procs, input_size in zip(num_procs_list[::2], input_sizes_list[2:]):
        for input_type in input_types:
            subset = data[
                (data['num_procs'] == num_procs) &
                (data['input_size'] == input_size) &
                (data['input_type'] == input_type)
                ]
            if not subset.empty:
                row = subset.iloc[0]
                weak_scaling_data.append({
                    'node': node.frame['name'],
                    'input_size': input_size,
                    'input_type': input_type,
                    'num_procs': num_procs,
                    'avg_time_per_rank': row['Avg time/rank']
                })

weak_df = pd.DataFrame(weak_scaling_data)
x_labels = []

for num_procs, input_size in zip(num_procs_list[::2], input_sizes_list[2:]):
    label = f'Procs: {num_procs}\nSize: {input_size}'
    x_labels.append(label)

for node_name in ['comp_large', 'comm', 'main']:
    for input_type in input_types:
        subset = weak_df[
            (weak_df['node'] == node_name) &
            (weak_df['input_type'] == input_type)
            ]
        if subset.empty:
            continue

        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=subset,
            x='num_procs',
            y='avg_time_per_rank',
            marker='o',
            label=f'Input Type: {input_type}'
        )

        plt.xscale('log', base=2)
        plt.xticks(ticks=num_procs_list[::2], labels=x_labels)
        plt.grid(True, which='both', linestyle='--', linewidth=0.7)
        plt.title(f'{algorithm}\nWeak Scaling: {node_name} - {input_type}')
        plt.xlabel('')
        plt.ylabel('Average Time per Rank (seconds)')
        plt.tight_layout()
        plt.savefig(f'./plots_report/weak_scaling_{node_name}_{input_type}.png')
        plt.close()
